In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
from scipy import stats
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from statsmodels.formula.api import ols

X_train = pd.read_csv('bakeoff_data/Xtrain.csv')
y_train = pd.read_csv('bakeoff_data/ytrain.csv')
X_test = pd.read_csv('bakeoff_data/Xtest.csv')


In [4]:
X_test['sqft_basement'] = X_test['sqft_basement'].apply(lambda x: np.nan if "?" in x else x)

In [5]:
X_train['sqft_basement'] = X_train['sqft_basement'].apply(lambda x: np.nan if "?" in x else x)

In [6]:
X_test_no_na = X_test.fillna({'waterfront': 0, 'view': 0, 'yr_renovated': 0, 'sqft_basement': 0})
X_train_no_na = X_train.fillna({'waterfront': 0, 'view': 0, 'yr_renovated': 0, 'sqft_basement': 0})

In [8]:
X_train_no_na['sqft_basement'] = X_train_no_na['sqft_basement'].astype(float)
X_test_no_na['sqft_basement'] = X_test_no_na['sqft_basement'].astype(float)

In [9]:
X_train_drop = X_train_no_na.drop(['sqft_above', 'grade', 'sqft_living15', 'bathrooms',\
                                   'date', 'long', 'lat', 'bedrooms'], axis = 1)
X_test_drop = X_test_no_na.drop(['sqft_above', 'grade', 'sqft_living15', 'bathrooms',\
                                 'date', 'long', 'lat', 'bedrooms'], axis = 1)

In [10]:
X_train_drop.isna().sum()

sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
sqft_lot15       0
dtype: int64

In [11]:
X_test_drop.isna().sum()

sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
sqft_lot15       0
dtype: int64

In [28]:
from sklearn.preprocessing import OneHotEncoder
zipcode = X_train_drop[['zipcode']]
ohe = OneHotEncoder(categories='auto', sparse=False, handle_unknown='ignore')
ohe.fit(zipcode)
ohe.categories_
zip_encoded = ohe.transform(zipcode)

zip_encoded = pd.DataFrame(

    zip_encoded,

    columns=ohe.categories_[0],
 
    index=X_train_drop.index
)

zip_encoded = zip_encoded.drop(98001, axis=1)
zip_encoded

,98002,98003,98004,98005,98006,98007,98008,98010,98011,98014,...,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
16194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [29]:
X_train_drop = X_train_drop.drop('zipcode', axis = 1)

In [30]:
X_train_drop = pd.concat([X_train_drop, zip_encoded], axis = 1)

In [31]:
zipcode_test = X_test_drop[['zipcode']]
ohe = OneHotEncoder(categories='auto', sparse=False, handle_unknown='ignore')
ohe.fit(zipcode)
ohe.categories_
zip_encoded_test = ohe.transform(zipcode_test)

zip_encoded_test = pd.DataFrame(

    zip_encoded_test,

    columns=ohe.categories_[0],
 
    index=X_test_drop.index
)
zip_encoded_test = zip_encoded_test.drop(98001, axis=1)

zip_encoded_test

,98002,98003,98004,98005,98006,98007,98008,98010,98011,98014,...,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
X_test_drop = X_test_drop.drop('zipcode', axis = 1)

In [33]:
X_test_drop = pd.concat([X_test_drop, zip_encoded_test], axis = 1)

In [48]:
X_test_drop

,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_basement,yr_built,yr_renovated,sqft_lot15,...,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
0,850,8573,1.0,0.0,0.0,3,250.0,1945,0.0,8382,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1510,6083,1.0,0.0,0.0,4,650.0,1940,0.0,5712,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1790,42000,1.0,0.0,0.0,3,620.0,1983,0.0,50094,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1140,2500,1.0,0.0,1.0,3,510.0,1988,0.0,5000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1500,3920,1.0,0.0,0.0,3,500.0,1947,0.0,4017,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,1170,6757,1.0,0.0,0.0,4,370.0,1944,0.0,6794,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5396,1670,5100,1.0,0.0,2.0,5,680.0,1954,0.0,4452,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5397,3260,4640,2.0,0.0,0.0,5,900.0,1907,0.0,5800,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5398,5820,13906,2.0,0.0,0.0,3,2070.0,1993,0.0,13000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Regression


In [34]:
from sklearn.linear_model import LinearRegression

In [35]:
lr = LinearRegression()
lr.fit(X_train_drop, y_train)
lr.score(X_train_drop,y_train)

0.7935611781637106

In [36]:
y_hat = lr.predict(X_test_drop)
y_hat.shape

(5400, 1)

In [37]:
from sklearn.model_selection import train_test_split

X_t, X_val, y_t, y_val = train_test_split(X_train_drop, y_train, random_state=42, test_size=.2)

In [38]:
lr.fit(X_t,y_t)
lr.score(X_t,y_t)

0.7964478757321444

In [39]:
lr.score(X_val, y_val)

0.7757965987490382

## Log Transform

In [40]:
lr_log = LinearRegression()
y_t_log = np.log(y_t)
y_val_log = np.log(y_val)
lr_log.fit(X_t, y_t_log)
lr_log.score(X_t,y_t_log)

0.8581526597430644

In [41]:
lr_log.score(X_val, y_val_log)

0.8389816508465457

## Final

In [42]:
lr_final = LinearRegression()
y_train_log = np.log(y_train)

lr_final.fit(X_train_drop, y_train_log)
lr_final.score(X_train_drop, y_train_log)

0.8547306351784946

In [43]:
y_hat_log = lr_final.predict(X_test_drop)


In [44]:
lr.fit(X_train_drop, y_train)

LinearRegression()

In [52]:
y_hat_log = lr_final.predict(X_test_drop)

In [53]:
y_hat = np.e ** y_hat_log

In [54]:
import numpy as np
y_test_fake = np.full((5400,1), 0)

In [55]:
from sklearn.metrics import r2_score

# fake predictions using the mean of y_train.
your_y_hat_predictions = np.full((5400,1), np.mean(y_train))

r2_score(y_hat, y_test_fake)

-1.0733684865162934

In [56]:
np.savetxt('example.csv', y_hat, delimiter=',')